# 🌲 Modelo 4: Regresión con Random Forest

Este notebook implementa un modelo de regresión usando **Random Forest Regressor** para predecir el `impact_score` de los jugadores en Rainbow Six Siege.

## Características del modelo:
- **Algoritmo**: Random Forest Regressor
- **Variable objetivo**: impact_score
- **Ventajas**: Reduce overfitting, maneja datos categóricos y numéricos, robusto a outliers
- **Desventajas**: Menos interpretable que un árbol individual, mayor complejidad computacional

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Cargar extensión de Kedro
%load_ext kedro.ipython

In [ ]:
# Cargar los datos
data = catalog.load("data_final")

print("📊 Información del dataset:")
print(f"Shape: {data.shape}")
print(f"Columnas disponibles: {len(data.columns)}")
print("\n🎯 Variable objetivo: impact_score")
print(f"Estadísticas del target:")
print(data['impact_score'].describe())

## 🔧 Preparación de los datos

Para Random Forest, utilizaremos las mismas características que en el árbol de decisión individual, pero el ensemble de múltiples árboles debería proporcionar mejor generalización y reducir el overfitting.

In [ ]:
# Preparar X e y (mismas variables que el árbol de decisión)
X = data[[ 
    'gamemode',        # tipo de partida
    'winrole',         # atacante o defensor
    'endroundreason',  # cómo terminó la ronda
    'roundduration',   # cuánto duró
    'isdead',          # si el jugador murió
    'nbkills',]]       # número de eliminaciones

y = data['impact_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo Random Forest
model = RandomForestRegressor(
    n_estimators=100,      # número de árboles en el bosque
    max_depth=10,          # profundidad máxima de cada árbol
    min_samples_split=20,  # mínimo de muestras para dividir un nodo
    min_samples_leaf=10,   # mínimo de muestras en una hoja
    random_state=42,       # para reproducibilidad
    n_jobs=-1             # usar todos los procesadores disponibles
)

model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Calcular métricas
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Mostrar resultados en formato simple
print("MSE: ", mse)
print("MAE: ", mae)
print("RMSE: ", rmse)
print("R^2: ", r2)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir el espacio de hiperparámetros
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [5, 10]
}

# Configurar el GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

# Ejecutar la búsqueda en los datos de entrenamiento
grid_search.fit(X_train, y_train)



# Evaluar el mejor modelo encontrado por GridSearchCV en el conjunto de test
best_model = grid_search.best_estimator_
y_pred_gs = best_model.predict(X_test)


mse_gs = mean_squared_error(y_test, y_pred_gs)
mae_gs = mean_absolute_error(y_test, y_pred_gs)
rmse_gs = mse_gs ** 0.5
r2_gs = r2_score(y_test, y_pred_gs)

# Mostrar que terminó el GridSearch y los resultados
print("GridSearchCV completado.")


In [ ]:
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

print("Resultados del mejor modelo tras GridSearchCV:")
print("MSE (test): ", mse_gs)
print("MAE (test): ", mae_gs)
print("RMSE (test): ", rmse_gs)
print("R^2 (test): ", r2_gs)